# WSTOFinalOrderQuantity plugin

input : 
Select ([Version].[Version Name] * [Item].[Item] * [Activity1].[Activity1] ) on row, 
({Measure.[PandoraSR Item Act1ToLoc Association]}) on column;

transformation : 
get [Item].[Item] * [Activity1].[Activity1]  where if(Measure.[PandoraSR Item Act1ToLoc Association]==1)
then Measure.[W STO Final Order Quantity] = 0.00000001

output measure : Measure.[W STO Final Order Quantity]

In [ ]:
# create spark session and import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col,lit,date_format, lpad, weekofyear, year
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .appName("Actual_To_SR_Actual") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/07 09:58:29 WARN Utils: Your hostname, codespaces-780a6a, resolves to a loopback address: 127.0.0.1; using 10.0.1.105 instead (on interface eth0)
25/08/07 09:58:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/07 09:58:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from o9_common_utils.O9DataLake import O9DataLake 
import logging 
from pyspark.sql.functions import concat, col,lit
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
logger = logging.getLogger("o9_logger") 

# o9datalake inputs

input_PandoraSR_Item_Act1ToLoc_Association  = O9DataLake.get("input_PandoraSR_Item_Act1ToLoc_Association ") 

In [4]:
#spark inputs
spark.sparkContext.setLogLevel("ERROR")
input_PandoraSR_Item_Act1ToLoc_Association = spark.read.csv("/workspaces/my_development_codes/data/PandoraSR*.csv", header=True, inferSchema=True)
input_PandoraSR_Item_Act1ToLoc_Association.show(5)

+----------------------+-----------+---------------------+------------------------------------+
|Version.[Version Name]|Item.[Item]|Activity1.[Activity1]|PandoraSR Item Act1ToLoc Association|
+----------------------+-----------+---------------------+------------------------------------+
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
+----------------------+-----------+---------------------+------------------------------------+
only showing top 5 rows


In [19]:
# i want to filter the output where column PandoraSR Item Act1ToLoc Association = 1
input_PandoraSR_Item_Act1ToLoc_Association = input_PandoraSR_Item_Act1ToLoc_Association.filter(
    col("PandoraSR Item Act1ToLoc Association") == 1
)
input_PandoraSR_Item_Act1ToLoc_Association.show(5)

currentday = spark.range(1).selectExpr("current_date() as today")
currentweek = currentday.select(
    concat(
        lit("W"),
        lpad(weekofyear("today").cast("string"), 2, "0"),
        lit("-"),
        year("today").cast("string")
    ).alias("week_label")
)

output_WSTOFinalOrderQuantity = input_PandoraSR_Item_Act1ToLoc_Association.select(
    lit('CurrentWorkingView').alias("Version.[Version Name]"),
    lit('EDC').alias("Location.[Location]"),col("`Item.[Item]`"),col("`Activity1.[Activity1]`"),
    lit(currentweek.collect()[0][0]).alias("Time.[Week]"),
    lit('TRUCK').alias("Activity2.[Activity2]"),col("`Item.[Item]`").alias("Activity3.[Activity3]"),
    lit('0.0000001').alias("W STO Final Order Quantity")
)
output_WSTOFinalOrderQuantity.show(5)

+----------------------+-----------+---------------------+------------------------------------+
|Version.[Version Name]|Item.[Item]|Activity1.[Activity1]|PandoraSR Item Act1ToLoc Association|
+----------------------+-----------+---------------------+------------------------------------+
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
|    CurrentWorkingView|  792352C01| LANE_From_EDC_To_...|                                   1|
+----------------------+-----------+---------------------+------------------------------------+
only showing top 5 rows
+----------------------+-------------------+-----------+---------------------+-----------+---------------------+

+----------------------+-------------------+-----------+---------------------+-----------+---------------------+---------------------+--------------------------+
|Version.[Version Name]|Location.[Location]|Item.[Item]|Activity1.[Activity1]|Time.[Week]|Activity2.[Activity2]|Activity3.[Activity3]|W STO Final Order Quantity|
+----------------------+-------------------+-----------+---------------------+-----------+---------------------+---------------------+--------------------------+
|    CurrentWorkingView|                EDC|  792352C01| LANE_From_EDC_To_...|   W32-2025|                TRUCK|            792352C01|                 0.0000001|
|    CurrentWorkingView|                EDC|  792352C01| LANE_From_EDC_To_...|   W32-2025|                TRUCK|            792352C01|                 0.0000001|
|    CurrentWorkingView|                EDC|  792352C01| LANE_From_EDC_To_...|   W32-2025|                TRUCK|            792352C01|                 0.0000001|
|    CurrentWorkingView|    